In [1]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from eipy.ei import EnsembleIntegration
import eipy.utils
from eipy.additional_ensembles import MeanAggregation, CES
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import  datasets

In [2]:
# If data is multi-class, run a check on the allowable base and meta models.

base_predictors = {
                    'ADAB': AdaBoostClassifier(),
                    'XGB': XGBClassifier(),
                    'DT': DecisionTreeClassifier(),
                    'RF': RandomForestClassifier(),
                    'GB': GradientBoostingClassifier(),
                    'KNN': KNeighborsClassifier(),
                    'LR': LogisticRegression(),
                    'NB': GaussianNB(),
                    'MLP': MLPClassifier(),
                    'SVM': SVC(probability=True)
}
natively_multi_class_predictors = ["XGBClassifier",
"BernoulliNB",
"DecisionTreeClassifier",
"ExtraTreeClassifier",
"GaussianNB",
"KNeighborsClassifier",
"LabelPropagation",
"LabelSpreading",
"LinearDiscriminantAnalysis",
"LinearSVC", #(setting multi_class=”crammer_singer”)
"LogisticRegression", #(setting multi_class=”multinomial”)
"LogisticRegressionCV", #(setting multi_class=”multinomial”)
"MLPClassifier",
"NearestCentroid",
"QuadraticDiscriminantAnalysis",
"RadiusNeighborsClassifier",
"RandomForestClassifier",
"RidgeClassifier",
"RidgeClassifierCV"]

In [3]:
base_predictors = {k : v for k,v in base_predictors.items() if str(v).split("(")[0] in natively_multi_class_predictors}
base_predictors

{'XGB': XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=None, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=None, n_jobs=None,
               num_parallel_tree=None, random_state=None, ...),
 'DT': DecisionTreeClassifier(),
 'RF': RandomForestClassifier(),
 'KNN': KNeighborsClassifier(),
 'LR': LogisticRegression(),
 'NB': GaussianNB(),
 'MLP': MLPClassifier()}

In [4]:
"""https://dev.pages.lis-lab.fr/scikit-multimodallearn/tutorial/auto_examples/combo/plot_combo_3_views_3_classes.html#
multi modal multi-class toy data generation"""

def generate_data(n_samples, lim):
    """Generate random data in a rectangle"""
    lim = np.array(lim)
    n_features = lim.shape[0]
    data = np.random.random((n_samples, n_features))
    data = (lim[:, 1]-lim[:, 0]) * data + lim[:, 0]
    return data
seed = 12
np.random.seed(seed)

n_samples = 300

modality_0 = np.concatenate((generate_data(n_samples, [[0., 1.], [0., 1.]]),
                         generate_data(n_samples, [[1., 2.], [0., 1.]]),
                         generate_data(n_samples, [[0., 2.], [0., 1.]])))

modality_1 = np.concatenate((generate_data(n_samples, [[1., 2.], [0., 1.]]),
                         generate_data(n_samples, [[0., 2.], [0., 1.]]),
                         generate_data(n_samples, [[0., 1.], [0., 1.]])))

modality_2 = np.concatenate((generate_data(n_samples, [[0., 2.], [0., 1.]]),
                         generate_data(n_samples, [[0., 1.], [0., 1.]]),
                         generate_data(n_samples, [[1., 2.], [0., 1.]])))

y = np.zeros(3*n_samples, dtype=np.int64)
y[n_samples:2*n_samples] = 1
y[2*n_samples:] = 2


In [5]:
X_0_train, X_0_test, y_train, y_test = train_test_split(modality_0, y, test_size=0.2, random_state=3, stratify=y)
X_1_train, X_1_test, _,_ = train_test_split(modality_1, y, test_size=0.2, random_state=3, stratify=y)
X_2_train, X_2_test, _,_ = train_test_split(modality_2, y, test_size=0.2, random_state=3, stratify=y)

In [6]:
data_train = {
                "Modality_0": X_0_train,
                "Modality_1": X_1_train,
                "Modality_2": X_2_train
                }

data_test = {
                "Modality_0": X_0_test,
                "Modality_1": X_1_test,
                "Modality_2": X_2_test
                }

In [17]:
EI = EnsembleIntegration(
                        base_predictors=base_predictors,
                        k_outer=5,
                        k_inner=5,
                        n_samples=1,
                        sampling_strategy=None,  #balanced classes
                        sampling_aggregation="mean",
                        n_jobs=-1,
                        random_state=42,
                        project_name="toy",
                        model_building=True,
                        )



In [18]:
for name, modality in data_train.items():
    EI.train_base(modality, y_train, modality_name=name)

Training base predictors on Modality_0...
        
... for ensemble performance analysis...


Generating meta training data: |          |  0%


ValueError: Found input variables with inconsistent numbers of samples: [720, 112]

In [16]:
EI.meta_training_data

[modality       Modality_0                                                \
 base predictor        XGB   DT    RF  KNN        LR        NB       MLP   
 sample                  0    0     0    0         0         0         0   
 0                0.044526  0.0  0.10  0.0  0.495549  0.423774  0.549981   
 1                1.757155  2.0  1.18  1.2  0.600471  0.511809  0.701635   
 2                1.378124  2.0  1.30  1.2  1.211018  1.346906  1.170763   
 3                1.088317  1.0  1.17  0.8  1.318650  1.494532  1.346982   
 4                1.535636  0.0  1.06  1.2  1.076944  0.927887  0.941663   
 ..                    ...  ...   ...  ...       ...       ...       ...   
 571              0.200107  0.0  0.28  0.4  0.359004  0.473228  0.559008   
 572              1.268022  0.0  0.37  0.4  0.938294  0.679345  0.797289   
 573              1.095236  1.0  1.29  1.2  1.171297  1.181220  1.230057   
 574              0.016178  0.0  0.32  0.4  0.457635  0.685845  0.644829   
 575        

In [10]:
iris = datasets.load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

In [11]:
base_predictors = {
                    'ADAB': AdaBoostClassifier(),
                    'XGB': XGBClassifier(),
                    'DT': DecisionTreeClassifier(),
                    'RF': RandomForestClassifier(),
                    'GB': GradientBoostingClassifier(),
                    'KNN': KNeighborsClassifier(),
                    'LR': LogisticRegression(),
                    'NB': GaussianNB(),
                    'MLP': MLPClassifier(),
                    'SVM': SVC(probability=True)
}
model = base_predictors["SVM"]
model.fit(X_train,y_train)
y_pred  = model.predict_proba(X_test)
y_pred[:5],y_pred[:, 1][:5]

#print(y_pred[:5])
#y_pred = [max(range(len(prob)), key=lambda i: prob[i]) for prob in y_pred] #change all probability vectors to predictions for scoring
#print(y_pred[:5])

(array([[0.00981237, 0.0276197 , 0.96256793],
        [0.00948382, 0.98086462, 0.00965155],
        [0.96882308, 0.0189461 , 0.01223082],
        [0.01033915, 0.00398004, 0.98568081],
        [0.96210977, 0.02622078, 0.01166945]]),
 array([0.0276197 , 0.98086462, 0.0189461 , 0.00398004, 0.02622078]))

In [12]:
from sklearn.metrics import precision_score

precision_score(y_test, y_pred, average='macro')

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets